In [ ]:
customer_order = """醤油ラーメンをください。野菜は大盛で、ニンニクも入れてください。
味は濃いめ、麺は固めでお願いします。
"""

order_template = """以下の注文情報に基づいて以下の情報を抽出して下さい。

ramen_type: ラーメンの種類
add_vegetable: 野菜の量
add_garlic: ニンニクの有無
taste: 味の濃さ
noodle: 麺の固さ

出力フォーマットはJSONで、以下の要素を持ちます。
ramen_type
add_vegetable
add_garlic
taste
noodle


注文情報: {order}
"""

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

ramen_type_schema = ResponseSchema(name="ramen_type",
                                        description="ラーメンの種類、指定が無い場合は醤油ラーメン")
add_vegetable_schema = ResponseSchema(name="add_vegetable",
                                        description="野菜の量、指定が無い場合は普通盛")
add_garlic_schema = ResponseSchema(name="add_garlic",
                                        description="ニンニクの有無、指定が無い場合は無")
taste_schema = ResponseSchema(name="taste",
                                        description="味の濃さ、指定が無い場合は普通")
noodle_schema = ResponseSchema(name="noodle",
                                        description="麺の固さ、指定が無い場合は普通")


response_schemas = [ramen_type_schema, add_vegetable_schema, add_garlic_schema, taste_schema, noodle_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(template=order_template)

messages = prompt.format_messages(order=customer_order, 
                                format_instructions=format_instructions)

In [ ]:
print(messages[0].content)

In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
from langchain.chat_models import AzureChatOpenAI

chat = AzureChatOpenAI(
    openai_api_base=os.getenv('AOAI_ENDPOINT'),
    openai_api_version="2023-05-15",
    deployment_name=os.getenv('AOAI_MODEL'),
    openai_api_key=os.getenv('AOAI_API_KEY'),
    openai_api_type = "azure",
    temperature=0.0,
)
response = chat(messages)

In [ ]:
print(response.content)

In [ ]:
output_dict = output_parser.parse(response.content)

In [ ]:
output_dict

In [ ]:
type(output_dict)

In [ ]:
output_dict.get('taste')